# Framework for predictions and portfolio forming

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time

# import the parquet library
import pyarrow.parquet as pq

# import model libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
# load 'basemodel.parquet'
#df = pd.read_parquet('basemodel.parquet')
df = pd.read_parquet('/kaggle/input/sign-prediction-datasets/basicmarket.parquet')
prediction_cols = []
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,STR,LTURNOVER,IMOM,MOM,LTR,PCTHIGH,IVOL2,IVOL3,MVOL,MVOL2,MVOL3,LMKT2,LMKT3,MMOM,MIMOM,MLTR,LSPREAD
0,10000,1987-03-31,-0.384615,973.25000,0.0,0.0,0.0,1.0,0,0,0.0482,0.000612,0,0.000000,0.100694,-1.089044,-2.079441,-0.297252,0.091549,0.000830,0.002413,0.545027,0.771919,0.434190,0.1289,-0.0278,0.186212,0.112764,0.459110,0.076923
1,10000,1987-04-30,-0.062500,912.44134,0.0,0.0,0.0,0.0,0,1,0.0211,0.003465,0,-0.384615,0.285384,-1.459321,-2.390877,0.014185,0.062500,0.000612,0.000830,0.672597,0.545027,0.771919,0.0482,0.1289,0.179935,-0.025601,0.498953,0.625000
2,10000,1987-05-31,-0.066667,851.59375,0.0,0.0,0.0,0.0,0,2,-0.0167,0.001893,0,-0.062500,0.256358,-1.633155,-2.772587,-0.089613,0.075378,0.003465,0.000612,1.565461,0.672597,0.545027,0.0211,0.0482,0.208747,0.032263,0.488026,0.133330
3,10001,1987-03-31,0.036800,6317.62500,0.0,0.0,0.0,1.0,0,0,0.0482,0.001627,1,-0.074074,0.368315,0.071156,0.140122,0.020203,0.892857,0.001556,0.001527,0.545027,0.771919,0.434190,0.1289,-0.0278,0.186212,0.112764,0.459110,0.120000
4,10001,1987-04-30,-0.039216,6069.87500,0.0,0.0,0.0,0.0,0,0,0.0211,0.001597,0,0.036800,0.217962,0.043187,0.038273,0.045090,0.910714,0.001627,0.001556,0.672597,0.545027,0.771919,0.0482,0.1289,0.179935,-0.025601,0.498953,0.078431


In [3]:
# select the columns to be used for prediction
X_col = ['bull_D', 'bear_D', 'bull_W', 'bear_W', 'bull_M', 'bear_M', 'LMKT', 'IVOL', # initial columns
         'STR',	'LTURNOVER', 'IMOM', 'MOM',	'LTR', 'PCTHIGH', 'IVOL2', 'IVOL3', 'LSPREAD',	# stock specific columns
         'MVOL', 'MVOL2', 'MVOL3', 'LMKT2',	'LMKT3', 'MMOM', 'MIMOM', 'MLTR'] # market specific columns

In [4]:
# scale the data for faster coeficient convergence
scaler = MinMaxScaler()

df[X_col] = scaler.fit_transform(df[X_col])

In [5]:
# Convert 'date' to datetime format (if not already done) and sort the DataFrame
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Create a 'year' column based on the 'date' column
df['year'] = df['date'].dt.year

# Paper Replication - OLS and Logit, Expanding Window - No Hyperparameters
- They start with out of sample forecasting in 1932
- models will be named model_default

### Linear Regression (Pooled OLS)

In [6]:
df.tail()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,STR,LTURNOVER,IMOM,MOM,LTR,PCTHIGH,IVOL2,IVOL3,MVOL,MVOL2,MVOL3,LMKT2,LMKT3,MMOM,MIMOM,MLTR,LSPREAD,year
3076047,20412,2022-07-31,-0.203046,4.174787e+04,0.166667,0.0,0.083333,0.0,0.000000,0.833333,0.304629,0.000074,0,0.030776,0.000056,0.575618,0.631158,0.707753,0.068333,0.000222,0.000118,0.109743,0.128462,0.073585,0.423071,0.288758,0.520342,0.538953,0.814156,0.009789,2022
3076048,16874,2022-07-31,0.135593,1.110135e+06,0.250000,0.0,0.166667,0.0,0.000000,0.000000,0.304629,0.000006,1,0.038978,0.000015,0.607527,0.717377,0.713835,0.811097,0.000034,0.000010,0.109743,0.128462,0.073585,0.423071,0.288758,0.520342,0.538953,0.814156,0.002150,2022
3076049,20395,2022-07-31,0.074970,3.728386e+06,0.000000,0.0,0.083333,0.0,0.166667,0.000000,0.304629,0.000005,1,0.040109,0.000054,0.604234,0.735304,0.708705,1.000000,0.000003,0.000004,0.109743,0.128462,0.073585,0.423071,0.288758,0.520342,0.538953,0.814156,0.001340,2022
3076050,16857,2022-07-31,-0.029348,1.304941e+05,0.000000,0.0,0.000000,0.0,0.000000,0.250000,0.304629,0.000071,0,0.038915,0.000081,0.524726,0.645115,0.780172,0.408233,0.000130,0.000202,0.109743,0.128462,0.073585,0.423071,0.288758,0.520342,0.538953,0.814156,0.004310,2022
3076051,93436,2022-07-31,0.323765,9.311106e+08,0.166667,0.0,0.083333,0.0,0.000000,0.000000,0.304629,0.000030,1,0.035278,0.000218,0.639349,0.729311,0.866215,0.587806,0.000033,0.000061,0.109743,0.128462,0.073585,0.423071,0.288758,0.520342,0.538953,0.814156,0.004467,2022


# My Experiments

## Machine Learning - Hyperparameter Tuning included in the process
- models to be named 'model_clas/reg_exp/roll'

### First expanding, then rolling
start predicting for 1932, expand the window until you reach X years, then roll it

#### MSE Evaluation

##### 5 years

In [7]:
rolling_window = 5

In [8]:
############################################
# LASSO CLASSIFICATION MODEL - MSE
############################################

model_name = 'lasso_clas_roll5_MSE'  # Name of the new column for storing predictions
start_time2 = time.time()  # Start timing


# Predefined set of C values for hyperparameter tuning
HP1 = [0.01, 0.1, 1, 10, 100] # C

# Update the column name for storing Decision Tree regression predictions
df[model_name] = np.nan

# Ensure the new column is in the prediction_cols list
if model_name not in prediction_cols:
    prediction_cols.append(model_name)

# Define the start year for modeling based on having at least 7 years of data
start_modeling_year = df['year'].min() + 6

for year in range(start_modeling_year, df['year'].max() + 1):
    start_time = time.time()  # Start timing
    
    # Determine the start year of the training window based on the current year
    train_start_year = max(year - rolling_window, df['year'].min())  # Ensure it does not go below the earliest year
    
    # Select the training data based on the calculated start year
    train_data = df[(df['year'] >= train_start_year) & (df['year'] < year)]
    
    # Split training data into actual training and tuning sets
    # Use the last year of the training data for tuning
    tuning_data = train_data[train_data['year'] == year - 1]
    actual_train_data = train_data[train_data['year'] < year - 1]
    
    X_train = actual_train_data[X_col]
    y_train = actual_train_data['y']
    
    X_tune = tuning_data[X_col]
    y_tune = tuning_data['y']
       
    best_HP1 = None
    best_mse = np.inf  # Initialize the best MSE to infinity
    
    # Hyperparameter tuning
    for hp1 in HP1:
        model = LogisticRegression(C=hp1, max_iter=1000, penalty='l1', solver='saga')  # Ridge
        model.fit(X_train, y_train)
        probabilities = model.predict_proba(X_tune)[:, 1]  # Get probabilities of the positive class
        mse = mean_squared_error(y_tune, probabilities)  # Calculate MSE
        
        if mse < best_mse:  # Lower MSE is better
            best_mse = mse
            best_HP1 = hp1
    
    
    # Retrain on the entire training window (excluding tuning year) with the best C value
    model = LogisticRegression(C=best_HP1, max_iter=1000, penalty='l1', solver='saga')
    model.fit(X_train, y_train)
    
    # Predict for the next year
    next_year_data = df[df['year'] == year]
    X_next_year = next_year_data[X_col]
    
    if not X_next_year.empty:
        next_year_probabilities = model.predict_proba(X_next_year)[:, 1]  # Probability of the positive class
        df.loc[df['year'] == year, model_name] = next_year_probabilities
    
    end_time = time.time()  # End timing
    iteration_time = end_time - start_time  # Calculate iteration time
    
    print(f"Year {year} - Best C: {best_HP1}, Best ACC: {round(best_mse,4)}, Time: {iteration_time:.2f} seconds")


end_time2 = time.time()  # End timing
print(f"Total time: {end_time2 - start_time2:.2f} seconds")

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 1933 - Best C: 100, Best ACC: 0.0673, Time: 65.47 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 1934 - Best C: 10, Best ACC: 0.0457, Time: 67.82 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 2021 - Best C: 1, Best ACC: 0.0244, Time: 436.07 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Year 2022 - Best C: 100, Best ACC: 0.0257, Time: 436.63 seconds
Total time: 31305.72 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Forming Portfolios, Value-weighted portfolio returns

In [9]:
df.head()

,PERMNO,date,RET,ME,bull_D,bear_D,bull_W,bear_W,bull_M,bear_M,LMKT,IVOL,y,STR,LTURNOVER,IMOM,MOM,LTR,PCTHIGH,IVOL2,IVOL3,MVOL,MVOL2,MVOL3,LMKT2,LMKT3,MMOM,MIMOM,MLTR,LSPREAD,year,lasso_clas_roll5_MSE
0,14314,1927-08-31,-0.164557,3902.25,0.0,0.000000,0.000000,0.083333,0.000000,0.0,0.538722,0.000033,0,0.040267,0.000003,0.551246,0.668133,0.709597,0.443203,0.000036,0.000097,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.002245,1927,NaN
1,12730,1927-08-31,-0.069853,88107.25,0.0,0.416667,0.000000,0.000000,0.000000,0.0,0.538722,0.000005,0,0.043893,0.000019,0.611572,0.736577,0.697120,0.824047,0.000005,0.000004,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.001956,1927,NaN
2,11594,1927-08-31,0.149390,14137.50,0.0,0.083333,0.000000,0.000000,0.250000,0.0,0.538722,0.000077,1,0.045470,0.000002,0.572206,0.709391,0.708355,0.869882,0.000005,0.000018,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.005677,1927,NaN
3,75471,1927-08-31,0.216216,1575.00,0.0,0.000000,0.000000,0.000000,0.083333,0.0,0.538722,0.000090,1,0.040866,0.000003,0.570800,0.709576,0.684379,0.860310,0.000038,0.000123,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.003594,1927,NaN
4,10786,1927-08-31,0.169811,111600.00,0.0,0.166667,0.583333,0.000000,0.083333,0.0,0.538722,0.000013,1,0.045594,0.000056,0.596852,0.722659,0.703022,1.000000,0.000003,0.000006,0.00106,0.008129,0.001942,0.397061,0.511976,0.645637,0.529397,0.644251,0.003555,1927,NaN


In [10]:
prediction_cols
# prediction_cols = ['logit_default','OLS_default','logit_roll6','DT_reg_roll']

['lasso_clas_roll5_MSE']

In [11]:
portfolio = df[['date', 'RET', 'ME', 'y'] + prediction_cols].copy()
portfolio['date'] = pd.to_datetime(portfolio['date'])

# drop rows with missing values
portfolio.dropna(inplace=True)

portfolio.head()

,date,RET,ME,y,lasso_clas_roll5_MSE
39215,1933-01-31,0.058824,456.750,1,8.189668e-01
39216,1933-01-31,0.427451,18036.000,1,1.000000e+00
39217,1933-01-31,-0.006993,1975.125,0,6.291103e-12
39218,1933-01-31,0.166667,241.500,1,1.000000e+00
39219,1933-01-31,-0.058824,3304.000,0,2.459881e-22


In [12]:
portfolio.tail()

,date,RET,ME,y,lasso_clas_roll5_MSE
3076047,2022-07-31,-0.203046,4.174787e+04,0,4.442020e-139
3076048,2022-07-31,0.135593,1.110135e+06,1,9.749049e-01
3076049,2022-07-31,0.074970,3.728386e+06,1,1.000000e+00
3076050,2022-07-31,-0.029348,1.304941e+05,0,6.178281e-43
3076051,2022-07-31,0.323765,9.311106e+08,1,9.995033e-01


In [13]:
# Initialize an empty DataFrame to store value-weighted returns for each model
vwreturns = pd.DataFrame(portfolio['date'].unique(), columns=['date'])  # Ensures all dates are included

for pred_col in prediction_cols:
    # Calculate deciles for this prediction
    decile_col = f'decile_{pred_col}'
    portfolio[decile_col] = portfolio.groupby(['date'])[pred_col].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))
    
    # Determine position based on deciles
    position_col = f'position_{pred_col}'
    portfolio[position_col] = np.where(portfolio[decile_col] == 9, 1, np.where(portfolio[decile_col] == 0, -1, 0))
    
    # Calculate the value-weighted return for this prediction
    vwret_col = f'vwreturn_{pred_col}'
    vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)
    
    # Merge the temporary value-weighted returns with the main vwreturns DataFrame
    vwreturns = vwreturns.merge(vwreturns_temp, on='date', how='left')

# Ensure the 'date' column is the first column and is sorted
vwreturns = vwreturns.sort_values('date').reset_index(drop=True)


/tmp/ipykernel_25/2999037948.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vwreturns_temp = portfolio.groupby('date').apply(lambda x: np.sum(x['RET'] * x['ME'] * x[position_col]) / np.sum(x['ME'])).reset_index(name=vwret_col)


In [14]:
vwreturns.head()

,date,vwreturn_lasso_clas_roll5_MSE
0,1933-01-31,0.024129
1,1933-02-28,0.009968
2,1933-03-31,0.020793
3,1933-04-30,0.110684
4,1933-05-31,-0.002718


### Compare to market data

In [15]:
#market = pd.read_csv('FF3_clean.csv')
market = pd.read_csv('/kaggle/input/sign-prediction-datasets/FF3_clean.csv')

In [16]:
market.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-31,2.96,-2.56,-2.43,0.22
1,1926-08-31,2.64,-1.17,3.82,0.25
2,1926-09-30,0.36,-1.40,0.13,0.23
3,1926-10-31,-3.24,-0.09,0.70,0.32
4,1926-11-30,2.53,-0.10,-0.51,0.31


In [17]:
# create a new 'Mkt' which is a sum of Mkt-RF and RF
market['Mkt'] = market['Mkt-RF'] + market['RF']

# divide all columns by 100 except 'date'
market.iloc[:, 1:] = market.iloc[:, 1:] / 100

#set the 'date' column to datetime format
market['date'] = pd.to_datetime(market['date'])

# merge the market data (only date and Mkt columns) with the vwreturns DataFrame
vwreturns = vwreturns.merge(market[['date', 'Mkt']], on='date', how='left')

# transform all columns (except 'date') to a log: log(x+1) and save the result as lvwreturns
lvwreturns = vwreturns.copy()
lvwreturns.iloc[:, 1:] = np.log(vwreturns.iloc[:, 1:] + 1)

In [18]:
vwreturns.head()

,date,vwreturn_lasso_clas_roll5_MSE,Mkt
0,1933-01-31,0.024129,0.0126
1,1933-02-28,0.009968,-0.1527
2,1933-03-31,0.020793,0.0333
3,1933-04-30,0.110684,0.3895
4,1933-05-31,-0.002718,0.2147


In [19]:
lvwreturns.head()

,date,vwreturn_lasso_clas_roll5_MSE,Mkt
0,1933-01-31,0.023842,0.012521
1,1933-02-28,0.009919,-0.165700
2,1933-03-31,0.020579,0.032758
3,1933-04-30,0.104976,0.328944
4,1933-05-31,-0.002722,0.194497


In [20]:
lvwreturns.describe()

,date,vwreturn_lasso_clas_roll5_MSE,Mkt
count,1072,1072.000000,1072.000000
mean,1977-11-17 20:54:37.611940288,0.005601,0.009039
min,1933-01-31 00:00:00,-0.027342,-0.272203
25%,1955-05-23 06:00:00,0.002006,-0.017248
50%,1977-12-15 12:00:00,0.003825,0.013262
75%,2000-04-07 12:00:00,0.007855,0.038235
max,2022-07-31 00:00:00,0.104976,0.328944
std,NaN,0.005961,0.048197


In [21]:
# save the lvwreturns and portfolio DataFrame to a parquet file into 'outputs' folder

# for reproducibility and visualization purposes
lvwreturns.to_parquet('market_lvwreturns_class2.parquet')
portfolio.to_parquet('market_portfolio_class2.parquet')

# save vwreturns DataFrame to a .dta file into 'outputs' folder
#vwreturns.to_stata('outputs/vwreturns.dta') # for backtasting in R - we need normal returns, not log returns
